In [ ]:
import pandas as pd
df = pd.read_pickle("/content/drive/My Drive/pinky_promise_last.pckl")

!pip install h5py

In [ ]:
df

,title,overview,genre_ids,text_vec,poster_path,image_vec
0,Travis and the Nitro Circus,"In January 2003, NC1 exploded on the scene and...","[28, 12]","[0.010523478500545025, 0.02272457629442215, 0....",/content/drive/My Drive/storage1/Travis_and_th...,[[[[ 0. 0. 0. 0. ...
1,Robot Chicken: Star Wars,"Fans of Adult Swim's ""Robot Chicken"" and the S...","[16, 35, 878]","[0.023902200162410736, 0.06350569427013397, -0...",/content/drive/My Drive/storage1/Robot_Chicken...,[[[[ 0. 0. 0. 0. ...
2,Sense8: Creating the World,Go behind the scenes and around the world with...,"[99, 878]","[0.00421142578125, 0.08410390466451645, 0.0145...",/content/drive/My Drive/storage1/Sense8:_Creat...,[[[[ 0. 0. 0. 0. ...
3,North of Nome,John Raglan is a seal hunter being hounded by ...,"[80, 12]","[0.01378631591796875, 0.051314543932676315, -0...",/content/drive/My Drive/storage1/North_of_Nome...,[[[[ 0. 0. 0. 0. ...
4,SPIT: The Story of a Caveman and a Chicken,A quirky caveman sets out on a journey to find...,"[35, 12]","[0.04422760009765625, 0.0597686767578125, -0.0...",/content/drive/My Drive/storage1/SPIT:_The_Sto...,[[[[ 0. 0. 0. 0. ...
...,...,...,...,...,...,...
42973,Get Lucky,Lucky is a small time criminal and he's happy ...,[28],"[0.0719434916973114, 0.052213314920663834, -0....",/content/drive/My Drive/storage3/Get_Lucky.jpg,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...
42974,Four Frightened People,Four passengers escape their bubonic plague-in...,"[12, 18, 53]","[0.08907948434352875, 0.14613474905490875, -0....",/content/drive/My Drive/storage3/Four_Frighten...,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...
42975,Iron Will,"When Will Stoneman's father dies, he is left a...",[12],"[0.0368289053440094, 0.0844523087143898, 0.013...",/content/drive/My Drive/storage3/Iron_Will.jpg,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...
42976,Bran Nue Dae,In the Summer of 1965 a young man is filled wi...,"[35, 18, 10749]","[0.08937211334705353, 0.08969070017337799, 0.0...",/content/drive/My Drive/storage3/Bran_Nue_Dae.jpg,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...


In [ ]:
import pickle

f1 = open("/content/drive/My Drive/Resnet_features_1.pckl","rb")
x1 = pickle.load(f1)
f2 = open("/content/drive/My Drive/Resnet_features_2.pckl","rb")
x2 = pickle.load(f2)
f3 = open("/content/drive/My Drive/Resnet_features_3.pckl","rb")
x3 = pickle.load(f3)

In [ ]:
X = x1[0]
X.extend(x2[0])
X.extend(x3[0])

In [ ]:
import numpy as np
feature_size=1000

np_features=np.zeros((len(X),feature_size))
for i in range(len(X)):
    feat=X[i]
    reshaped_feat=feat.reshape(1,-1)
    np_features[i]=reshaped_feat

X=np_features

In [ ]:
X.shape

(42978, 1000)

In [ ]:
f4 = open("/content/drive/My Drive/glove_features.pckl","rb")
glove = pickle.load(f4)
g = []
for i in range(len(glove[0])):
  g.append(glove[0][i])

In [ ]:
resnet_glove = np.zeros((42978, 1300))

In [ ]:
import numpy as np

for i in range(len(df)):
  x1 = g[i].flatten()
  x2 = X[i].flatten()
  x3 = np.concatenate((x1,x2),axis=0)
  resnet_glove[i] = x3
  if i%5000 == 0:
    print("count is ", i)

count is  0
count is  5000
count is  10000
count is  15000
count is  20000
count is  25000
count is  30000
count is  35000
count is  40000


In [ ]:
resnet_glove[0].shape

(1300,)

In [ ]:
Y = df.genre_ids.to_list()

In [ ]:
mask = np.random.rand(len(X)) < 0.8

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb=MultiLabelBinarizer()
Y=mlb.fit_transform(Y)

Y.shape

(42978, 14)

In [ ]:
X_train=resnet_glove[mask]
X_test=resnet_glove[~mask]
Y_train=Y[mask]
Y_test=Y[~mask]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
model_visual = Sequential([
    Dense(1024, input_shape=(1300,)),
    Activation('relu'),
    Dense(256),
    Activation('relu'),
    Dense(14),
    Activation('sigmoid'),
])
opt = optimizers.rmsprop(lr=0.0001, decay=1e-6)

#sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.4, nesterov=False)
model_visual.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

Using TensorFlow backend.


In [ ]:
model_visual.fit(X_train, Y_train, epochs=20, batch_size=64,verbose=1)

Epoch 1/20
34295/34295 [==============================] - 8s 234us/step - loss: 0.5175 - accuracy: 0.7250
Epoch 2/20
34295/34295 [==============================] - 8s 224us/step - loss: 0.4759 - accuracy: 0.7562
Epoch 3/20
34295/34295 [==============================] - 8s 225us/step - loss: 0.4633 - accuracy: 0.7656
Epoch 4/20
34295/34295 [==============================] - 8s 224us/step - loss: 0.4557 - accuracy: 0.7714
Epoch 5/20
34295/34295 [==============================] - 8s 225us/step - loss: 0.4499 - accuracy: 0.7749
Epoch 6/20
34295/34295 [==============================] - 8s 224us/step - loss: 0.4451 - accuracy: 0.7780
Epoch 7/20
34295/34295 [==============================] - 8s 227us/step - loss: 0.4407 - accuracy: 0.7821
Epoch 8/20
34295/34295 [==============================] - 8s 228us/step - loss: 0.4369 - accuracy: 0.7847
Epoch 9/20
34295/34295 [==============================] - 8s 225us/step - loss: 0.4330 - accuracy: 0.7867
Epoch 10/20
34295/34295 [=====================

In [ ]:
def precision_recall(gt,preds):
    TP=0
    FP=0
    FN=0
    for t in gt:
        if t in preds:
            TP+=1
        else:
            FN+=1
    for p in preds:
        if p not in gt:
            FP+=1
    if TP+FP==0:
        precision=0
    else:
        precision=TP/float(TP+FP)
    if TP+FN==0:
        recall=0
    else:
        recall=TP/float(TP+FN)
    return precision,recall

In [ ]:
Y_preds=model_visual.predict(X_test)

In [ ]:
import pickle
f6=open('/content/drive/My Drive/Genredict.pckl','rb')
Genre_ID_to_name=pickle.load(f6)
f6.close()

In [ ]:
genre_list=sorted(list(Genre_ID_to_name.keys()))

In [ ]:
sum(sum(Y_preds))

66936.46423339844

In [ ]:
import numpy as np
positions=range(len(df))
# print positions
test_movies=np.asarray(positions)[~mask]
# test_movies

In [ ]:
import numpy as np

precs=[]
recs=[]
for i in range(len(Y_preds)):
    row=Y_preds[i]
    gt_genres=Y_test[i]
    gt_genre_names=[]
    for j in range(14):
        if gt_genres[j]==1:
            gt_genre_names.append(Genre_ID_to_name[genre_list[j]])
    top_3=np.argsort(row)[-4:]
    predicted_genres=[]
    for genre in top_3:
        predicted_genres.append(Genre_ID_to_name[genre_list[genre]])
    (precision,recall)=precision_recall(gt_genre_names,predicted_genres)
    precs.append(precision)
    recs.append(recall)
    print("Predicted: ",','.join(predicted_genres)," Actual: ",','.join(gt_genre_names))

Streaming output truncated to the last 5000 lines.
Predicted:  Fantasy,Adventure,Science Fiction,Mystery  Actual:  Adventure,Fantasy,Drama,Horror,Comedy,Thriller,Crime,Science Fiction,Mystery
Predicted:  Drama,Crime,Mystery,Science Fiction  Actual:  Horror,Thriller,Science Fiction,Mystery
Predicted:  Drama,Documentary,Science Fiction,Mystery  Actual:  Adventure,Fantasy,Action,History,Western,Science Fiction,Mystery
Predicted:  Adventure,Drama,Mystery,Science Fiction  Actual:  Adventure,Fantasy,Animation,Drama,Action,History,Thriller,Crime,Science Fiction,Mystery
Predicted:  Drama,Horror,Science Fiction,Mystery  Actual:  Animation,Drama,Horror,Comedy,Science Fiction,Mystery
Predicted:  Action,Thriller,Mystery,Science Fiction  Actual:  Action,Thriller,Science Fiction,Mystery
Predicted:  Thriller,Action,Science Fiction,Mystery  Actual:  Adventure,Fantasy,Animation,Drama,Action,Comedy,History,Thriller,Documentary,Science Fiction,Mystery
Predicted:  Horror,Thriller,Science Fiction,Mystery  

In [ ]:
print(np.mean(np.asarray(precs)),np.mean(np.asarray(recs)))
print("avg=",(np.mean(np.asarray(precs))+np.mean(np.asarray(recs)))/2)

0.9044109178855234 0.5317856574305959
avg= 0.7180982876580597


In [ ]:
print("f1-score", (2*(np.mean(np.asarray(precs)) * np.mean(np.asarray(recs)))) / (np.mean(np.asarray(precs)) + np.mean(np.asarray(recs))))

f1-score 0.6697589491874396


In [ ]:
from keras.models import model_from_json

model_json = model_visual.to_json()
with open("Resnet_Glove.json", "w") as json_file:
    json_file.write(model_json)
model_visual.save_weights("Resnet_Glove.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
img = image.load_img('/content/drive/My Drive/storage3/Iron_Man.jpg',target_size=(224,224))
img = image.img_to_array(img)
img = img/255

In [ ]:
print(np.mean(np.asarray(precs)),np.mean(np.asarray(recs)))
print("avg=",(np.mean(np.asarray(precs))+np.mean(np.asarray(recs)))/2)

0.9034109178855234 0.5387856574305959
avg= 0.7210982876580596
